# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846877515916                   -0.70    4.8         
  2   -7.852325060555       -2.26       -1.53    1.0   29.9ms
  3   -7.852617180954       -3.53       -2.56    1.5   33.7ms
  4   -7.852645982539       -4.54       -2.89    2.5   39.5ms
  5   -7.852646515705       -6.27       -3.21    1.0   29.7ms
  6   -7.852646680042       -6.78       -4.21    1.2   31.3ms
  7   -7.852646686681       -8.18       -5.01    2.5   40.5ms
  8   -7.852646686727      -10.34       -5.61    1.5   33.2ms
  9   -7.852646686728      -11.82       -5.65    1.2    126ms
 10   -7.852646686730      -11.87       -6.89    1.0   29.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846796056623                   -0.70           4.8         
  2   -7.852524428549       -2.24       -1.63   0.80    2.0    298ms
  3   -7.852635844107       -3.95       -2.71   0.80    1.0   28.5ms
  4   -7.852646413953       -4.98       -3.27   0.80    2.2   36.4ms
  5   -7.852646673526       -6.59       -4.08   0.80    1.2   29.3ms
  6   -7.852646686431       -7.89       -4.67   0.80    1.8   34.1ms
  7   -7.852646686722       -9.54       -5.56   0.80    1.5   31.3ms
  8   -7.852646686730      -11.13       -6.81   0.80    2.0   33.6ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.376522e+01     3.273739e+00
 * time: 0.6213669776916504
     1     1.163086e+00     2.015866e+00
 * time: 0.8521590232849121
     2    -1.291250e+00     2.488249e+00
 * time: 0.8814489841461182
     3    -3.848991e+00     1.922931e+00
 * time: 0.9234428405761719
     4    -5.079111e+00     1.788699e+00
 * time: 0.9653220176696777
     5    -6.831459e+00     8.197735e-01
 * time: 1.007126808166504
     6    -7.435325e+00     3.498570e-01
 * time: 1.048750877380371
     7    -7.664920e+00     1.746317e-01
 * time: 1.0775949954986572
     8    -7.758537e+00     1.532436e-01
 * time: 1.1066968441009521
     9    -7.802714e+00     1.174758e-01
 * time: 1.135854959487915
    10    -7.829526e+00     7.852642e-02
 * time: 1.164902925491333
    11    -7.841253e+00     4.417699e-02
 * time: 1.193199872970581
    12    -7.847744e+00     3.557171e-02
 * time: 1.2220768928527832
    13    -7.850799e+00     2.516393e-02
 * time: 1.250910997390747

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846821610930                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645874135                   -1.64         
  2   -7.852646686730       -6.09       -3.71    1.93s
  3   -7.852646686730      -13.21       -7.21    208ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.1513621980340427e-7
|ρ_newton - ρ_scfv| = 1.3814689553878664e-7
|ρ_newton - ρ_dm|   = 1.1490597299993771e-9
